In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import descarteslabs as dl
import geopandas as gpd

from scripts import deploy_query_patch

In [ ]:
output_ids = []
roi_list = ['africa_new']
date_list = [['2021-01-01', '2022-07-01']]
for roi in roi_list:
    for dates in date_list:
        start_date = dates[0]
        end_date = dates[1]
        patch_version = '3.0'
        pred_threshold = '0.5'
        patch_product_name = f'earthrise:{roi}_patch_weak_labels_{patch_version}_{start_date}_{end_date}_stride_8'

        pixel_version = '0.0.11'
        pred_threshold = '0.6'
        min_sigma = '5.0'
        band = 'median'
        #pixel_product_name = f"{roi}_v{pixel_version}_{start_date}_{end_date}_blobs_thresh_{pred_threshold}_min-sigma_{min_sigma}_area-thresh_0.0025_band-{band}"
        pixel_product_name = f'earthrise:{roi}_v{pixel_version}_{start_date}_{end_date}'
        output_id = f"earthrise:intersect_{pixel_product_name.split('earthrise:')[-1]}-{patch_product_name.split('earthrise:')[-1]}_threshold_{pred_threshold}.geojson"
        output_ids.append(output_id)
        run_local = False

        args = [
            '--roi_name', roi,
            '--pixel_product_name', pixel_product_name,
            '--pixel_version', pixel_version,
            '--patch_product_name', patch_product_name,
            '--patch_version', patch_version,
            '--pred_threshold', pred_threshold
        ]
        if run_local:
            args.append('--run_local')
        
        # Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
        %cd ../scripts
        %pwd

        deploy_query_patch.main(args)

## Download data once task is complete

In [ ]:
for roi in roi_list:
    for output_id in output_ids:
        roi_file = f'../data/boundaries/{roi}.geojson'
        region = gpd.read_file(roi_file)['geometry']

        fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if output_id in fc.id][0]
        fc = dl.vectors.FeatureCollection(fc_id)

        features = []
        for i, r in enumerate(region):
            print("region", i)
            candidates = fc.filter(r).features()
            for elem in candidates:
                features.append(elem.geojson)
        candidate_gdf = gpd.GeoDataFrame.from_features(features)
        directory = f"../data/model_outputs/candidate_sites/{pixel_version}"
        candidate_gdf.to_file(os.path.join(directory, output_id.split(':')[-1] + '.geojson'))
        